In [1]:
#change working directory, for moduls of other package
import os
os.chdir(os.path.abspath('../../src'))

In [2]:
from mcts.mcts import Builder
from mcts.expansion import expansion_random_sampling, expansion_random
from environment.environment import SokobanEnvImpl
from environment.visualization import render
from environment.util import breadth_first_search
import random
import csv

In [3]:
random.seed(42)

In [4]:
solver_qwen = Builder().build()
solver_deepseek = Builder().build()
solver_chatgpt = Builder().build()
random_sampling_solver = Builder().setExpansion(expansion_random_sampling).build()
random_solver = Builder().setExpansion(expansion_random).build()

def write_to_summary(row):
    with open('../docs/comparisons/output/summary.csv', mode='a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['id', 'best_trajectory', 'qwen_trajectory', 'qwen_nodes', 'qwen_time', 'deepseek_trajectory', 'deepseek_nodes', 'deepseek_time', 'chatgpt_trajectory', 'chatgpt_nodes', 'chatgpt_time', 'rs_trajectory', 'rs_nodes', 'rs_time', 'r_trajectory', 'r_nodes', 'r_time'])
        # If the file is empty, write the header first
        if f.tell() == 0:
            writer.writeheader()
        writer.writerow(row)

def solve(env:SokobanEnvImpl, env_name:str):
    os.mkdir(f"../docs/comparisons/output/{env_name}")
    render(env, save_fig=f"../docs/comparisons/output/{env_name}/environment.png", show_fig=False)
    best_trajectory = breadth_first_search(env)
    qwen_trajectory, qwen_nodes, qwen_time = solver_qwen.solve(env, log_path=f"../docs/comparisons/output/{env_name}/qwen_", agent_player_model="qwen3:32b")
    deepseek_trajectory, deepseek_nodes, deepseek_time = solver_deepseek.solve(env, log_path=f"../docs/comparisons/output/{env_name}/deepseek_", agent_player_model="deepseek-r1:70b")
    chatgpt_trajectory, chatgpt_nodes, chatgpt_time = solver_chatgpt.solve(env, log_path=f"../docs/comparisons/output/{env_name}/chatgpt_", agent_player_model="gpt-oss:120b")
    rs_trajectory, rs_nodes, rs_time = random_sampling_solver.solve(env, log_path=f"../docs/comparisons/output/{env_name}/rs_")
    r_trajectory, r_nodes, r_time = random_solver.solve(env, log_path=f"../docs/comparisons/output/{env_name}/r_")
    write_to_summary({'id': env_name, 'best_trajectory': best_trajectory, 
                      'qwen_trajectory': qwen_trajectory, 'qwen_nodes': qwen_nodes, 'qwen_time': qwen_time, 
                      'deepseek_trajectory': deepseek_trajectory, 'deepseek_nodes': deepseek_nodes, 'deepseek_time': deepseek_time, 
                      'chatgpt_trajectory': chatgpt_trajectory, 'chatgpt_nodes': chatgpt_nodes, 'chatgpt_time': chatgpt_time, 
                      'rs_trajectory': rs_trajectory, 'rs_nodes': rs_nodes, 'rs_time': rs_time, 
                      'r_trajectory': r_trajectory, 'r_nodes': r_nodes, 'r_time': r_time})

In [5]:
env_name = "".zfill(4) + "_default_env"
env = SokobanEnvImpl(max_steps=60, use_default_env=True)
solve(env, env_name)

In [ ]:
for i in range(1, 51):
    env_name = str(i).zfill(4) + "_generated_env"
    env = SokobanEnvImpl(max_steps=60).as_fixated()
    solve(env, env_name)